In [1]:
from pyspark import SparkContext


In [2]:
sc = SparkContext()

In [4]:
rev = lambda s:s[::-1]

In [13]:
rev('abcde')

'edcba'

In [33]:
%%writefile example.txt
first line
second line
third line
fourth line

Overwriting example.txt


In [34]:
sc.textFile('example.txt')

example.txt MapPartitionsRDD[14] at textFile at NativeMethodAccessorImpl.java:0

In [35]:
text_rdd = sc.textFile('example.txt')

In [36]:
text_rdd.count()

4

In [37]:
text_rdd.first()

'first line'

In [38]:
secfind = text_rdd.filter(lambda line: 'second' in line)

In [39]:
secfind

PythonRDD[19] at RDD at PythonRDD.scala:53

In [40]:
secfind.collect()

['second line']

In [41]:
secfind.count()

1

In [42]:
words = text_rdd.map(lambda line:line.split())

In [43]:
words.collect()

[['first', 'line'], ['second', 'line'], ['third', 'line'], ['fourth', 'line']]

In [44]:
text_rdd.collect()

['first line', 'second line', 'third line', 'fourth line']

In [45]:
text_rdd.flatMap(lambda line:line.split()).collect()

['first', 'line', 'second', 'line', 'third', 'line', 'fourth', 'line']

In [46]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing services.txt


In [47]:
services = sc.textFile('services.txt')

In [48]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [50]:
services.map(lambda line: line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [53]:
clean = services.map(lambda line: line[1:] if line[0]=='#' else line)

In [54]:
clean = clean.map(lambda line: line.split())

In [55]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [57]:
pairs = clean.map(lambda lst: (lst[3], lst[-1])) #Step 1

In [58]:
pairs.collect()

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

In [67]:
rekey = pairs.reduceByKey(lambda amt1, amt2 : float(amt1)+float(amt2)) #Step 2

In [65]:
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [70]:
notitle = rekey.filter(lambda x: not x[0]=='State') #step 3

In [72]:
notitle.collect()

[('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [75]:
rdd_sorted = notitle.sortBy(lambda stAmount: stAmount[1], ascending=False) #step 4

In [76]:
rdd_sorted.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [77]:
x = ['ID','State', 'Amount']

In [78]:
def func1(lst):
    return lst[-1]

In [79]:
def func2(id_st_amt):
    #unpack values
    (Id, st, amt) = id_st_amt

In [80]:
func1(x)

'Amount'

In [82]:
func2(x)